#Embarcation pour l'univers de l'IA : le Titanic

01. Pourquoi sommes-nous ici ?
Aujourd'hui, l'Intelligence Artificielle est partout, mais elle est souvent perçue comme une "boîte noire" mystérieuse. Pour comprendre les outils révolutionnaires d'aujourd'hui (comme ChatGPT ou la reconnaissance faciale) il est indispensable de revenir aux fondations.

Les modèles que nous allons construire aujourd'hui, l'arbre de décision et la forêt aléatoire, ne sont pas de simples exercices de style : ils sont la base absolue de la science des données. Ils représentent la façon par laquelle on commence à apprendre aux machines à extraire de la logique à partir du "chaos" (données structurées, bruyantes).

02. Le changement de paradigme

Dans l'informatique "classique" pour résoudre un problème, on donne à l'ordinateur des règles explicites :

Exemple : SI (Le message contient "Gagner de l'argent") ET (L'expéditeur est inconnu) ALORS (Marquer comme SPAM)

En Machine Learning, on inverse la logique. On ne donne plus de règles à l'ordinateur. On lui donne :

Les données du passé (le profil des passagers du Titanic).

Les résultats connus (qui a survécu, qui est décédé).

C'est la machine qui crée ses propres règles. C'est ce qu'on appelle l'entraînement d'un modèle.

03. Des règles explicites aux règles implicites

La grande force des modèles que nous étudions aujourd'hui est leur transparence.
- Aujourd'hui (arbre de décision) : Les règles trouvées par l'IA sont explicites. On peut les dessiner et les lire facilement
- Plus tard (Modèles avancés/Deep Learning) : Dans des modèles plus complexes (réseaux de neurones) les règles deviennent implicites. L'IA "sait" mais elle utilise des millions de calculs mathématiques cachés que l'humain ne peut plus lire à l'œil nu.

Apprendre sur le jeu de données du Titanic c'est apprendre à lire dans les pensées de la machine avant qu'elles ne deviennent trop complexes.

04. Pourquoi le Titanic comme terrain de jeu ?

Le naufrage du Titanic en 1912 n'est pas qu'une tragédie historique ; c'est un laboratoire sociologique parfait. Les données sont "bruitées" (il y a des erreurs, des oublis) mais elles contiennent des structures sociales fortes.

Si vous arrivez à prédire la survie d'un passager avec une bonne précision, vous aurez maîtrisé les étapes fondamentales de n'importe quel projet d'IA :

- Préparer les données (le nettoyage). Ne sous-estimez **jamais** cette étape.

- Transformer le réel en mathématiques (l'encodage) pour l'interprétation de la machine.

- Entraîner et évaluer la logique résultante (l'apprentissage).

#0. Imports nécessaires

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Chargement des données directement depuis un dépôt GitHub public
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv"
df = pd.read_csv(url)

In [ ]:
print("Données chargées avec succès !")
df.head()

#1. Exploration des données

In [ ]:
# Traduction des colonnes pour faciliter la compréhension
traductions = {
    'PassengerId': 'ID_Passager',
    'Survived': 'Survécu',
    'Pclass': 'Classe',
    'Name': 'Nom',
    'Sex': 'Sexe',
    'Age': 'Age',
    'SibSp': 'Frères_Sœurs_Époux',
    'Parch': 'Parents_Enfants',
    'Ticket': 'Numéro_Ticket',
    'Fare': 'Prix_Billet',
    'Cabin': 'Cabine',
    'Embarked': 'Port_Embarquement'
}

In [ ]:
# Afficher les informations générales (colonnes, types, valeurs manquantes)
df.info()

In [ ]:
df = df.rename(columns=traductions)

print("En-têtes traduits !")
df.head()

In [ ]:
#@title Statistiques descriptives du jeu de données
# On regarde les moyennes, minimums, maximums
# On transpose (T) pour que ce soit plus lisible si on a beaucoup de colonnes
df.describe().T

In [ ]:
#@title Où sont les "trous" dans nos données ?
plt.figure(figsize=(10,6))
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.title("Lignes jaunes = Données manquantes")
plt.show()

print(df.isnull().sum())

In [ ]:
#@title Survie selon la classe à bord (1ère, 2ème, 3ème)
sns.barplot(x='Classe', y='Survécu', data=df)
plt.title("Probabilité de survie par classe (corrélation)")
plt.show()

In [ ]:
#@title Qui a survécu, selon le sexe ?
sns.countplot(x='Survécu', hue='Sexe', data=df)
plt.title("Survie selon le sexe (0 = Décédé, 1 = Survécu)")
plt.show()

In [ ]:
#@title Quel était l'âge des gens à bord ?
plt.figure(figsize=(10,5))
sns.histplot(df['Age'].dropna(), kde=True, color='blue', bins=30)
plt.title("Répartition des passagers par âge")
plt.show()

In [ ]:
#@title Bilan du naufrage : Survivants vs Décédés
plt.figure(figsize=(7, 5))

# Création du graphique en barres
sns.countplot(x='Survécu', data=df, hue='Survécu', palette=['#e74c3c', '#2ecc71'], legend=False)

# Personnalisation pour que ce soit clair pour les élèves
plt.title("Répartition des passagers (La réalité du naufrage)")
plt.xticks([0, 1], ['Décédés (0)', 'Survivants (1)'])
plt.xlabel("")
plt.ylabel("Nombre de passagers")

plt.show()

# Affichage des chiffres exacts
counts = df['Survécu'].value_counts()
print(f"Nombre de décès : {counts[0]}")
print(f"Nombre de survivants : {counts[1]}")
print(f"Pourcentage de survivants : {df['Survécu'].mean()*100:.2f}%")

#2. Nettoyage des données

Ici, deux points sont à retenir :
- Les données ne sont peut-être pas propres (erreurs, mauvais type, cellules vides, etc.)
- Le système d'apprentissage ne peut pas interpréter les mots en tant que tels, il faut les convertir en valeurs numériques

In [ ]:
#1. Remplir les âges manquants par la moyenne (Imputation) ou supprimer ces colonnes
print(f"Nombre de cellules null dans la colonne Age : {df["Age"].isna().sum()}")

In [ ]:
df[df['Age'].isna()]

In [ ]:
#On peut remplacer la cellule vide par l'âge médian des passagers
#df['Age'] = df['Age'].fillna(df['Age'].median())

##2.1. Vérification de la qualité des données

In [ ]:
#Ou supprimer les lignes où l'âge est vide en perdant donc des données
#Je vous conseille d'alterner pour observer l'impact sur la précision de vos modèles
df.dropna(subset=['Age'], inplace=True)

In [ ]:
df.info()

##2.2. Modification des données textuelles

In [ ]:
df.head()

## Question :
Quelles colonnes sont à transformer selon-vous ?

<details>
<summary>👉 Cliquez ici pour voir la réponse</summary>

**Réponse :** Seul le sexe est facile à faire et pertinent. Le nom, le numéro du ticket et la cabine sont des identifiants **uniques** donc peu utiles pour notre exercice. Le port d'embarquement pourrait être intéressant, vous pourrez tester par vous-même
</details>

In [ ]:
# 2. Transformer la colonne Sex en valeurs numériques (0 pour homme, 1 pour femme par ex.)
df['Sexe'] = df['Sexe'].map({'male': 0, 'female': 1})

In [ ]:
df.head()

##2.3. Simplification pour l'exercice

Ne gardons que les colonnes les plus explicites ET on sépare la valeur "survécu" des paramètres.

In [ ]:
features = ['Classe', 'Sexe', 'Age', 'Frères_Sœurs_Époux', 'Parents_Enfants', 'Prix_Billet']
X = df[features]
y = df['Survécu']

print("Données prêtes pour l'entraînement !")

In [ ]:
X.head()

In [ ]:
y.head()

#3. Création de jeux d'entrainement

Pourquoi jeux au pluriel ? Car il faut toujours, au moins, un jeu dédié à l'entrainement du modèle et un jeu d'évaluation de la qualité de la prédiction du modèle. Un découpage de 20% du jeu total est généralement recommandé pour le jeu d'évaluation.

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Nombre de passagers pour l'entraînement : {len(X_train)}") #en comptant les lignes dans le jeu, on obtient le nombre de passagers conservés car une ligne = un passager
print(f"Nombre de passagers pour le test : {len(X_eval)}")

#4. Entrainement d'un modèle

Ici, vous devez faire des choix / expérimentations concernant le modèle le plus adapté à vos données et votre tâche.

Choisissons un arbre de décision ici, l'un des modèles les plus simples afin de voir ce qu'il se passe lors de l'apprentissage.

In [ ]:
# Création du modèle
model = DecisionTreeClassifier(max_depth=3) # Vous pouvez faire varier la profondeur pour évaluer l'impact sur les résultats

# Apprentissage
model.fit(X_train, y_train)

print("Le modèle a terminé son apprentissage.")

#5. Prédictions avec le modèle

Une fois le modèle entrainé, nous pouvons l'utiliser pour prédire et donc dans un premier temps, évaluer son apprentissage.

In [ ]:
# On demande au modèle de prédire sur les données qu'il n'a jamais vues : le jeu d'évaluation
predictions = model.predict(X_eval)

# Calcul du score de précision
score = accuracy_score(y_eval, predictions) #on compare les prédictions aux formations réelles
print(f"Précision du modèle : {score * 100:.2f}%")

## Question : Pourquoi le score est-il meilleur sur la colonne "Décédé" ?

<details> <summary>👉 Réponse et analyse</summary>

Généralement, sur le Titanic, les modèles ont un meilleur score (Précision et Rappel) pour les décédés. Pourquoi ? Parce qu'il y a plus d'exemples de personnes décédées dans les données d'entraînement, tout simplement. Le modèle a donc plus de données (et de variations) pour reconnaître un passager qui ne va pas s'en sortir que pour reconnaître un survivant. C'est ce qu'on appelle un jeu de données déséquilibré.

</details>

In [ ]:
# Définir la taille du graphique
plt.figure(figsize=(20,10))

# Tracer l'arbre
plot_tree(model,
          feature_names=features,
          class_names=['Décédé', 'Survécu'],
          filled=True,
          rounded=True,
          fontsize=12)

plt.title("L'arbre de décision généré par le modèle lors de son apprentissage")
plt.show()

Chaque bloc est une décision à partir des informations données. Pour chaque bloc :
- Première ligne : condition selon laquelle on suit la branche True ou False. Pour la première case, c'est si l'age est inférieur ou égal à 0.5. H=0, F=1 donc "si non femme, True". C'est notre facteur le plus déterminant.
- Deuxième ligne gini : doit tendre vers 0 pour que le modèle soit sûr de lui.
- Troisième ligne : le nombre de passagers restants dans le groupe
- Quatrième ligne : nb décès, nb survécu dans le groupe
- Cinquième ligne : réponse liée à la prédiction / case dans l'arbre (Ex : Homme avec age <= 3.5, probablement décédé)

## Question :
Que se passe-t-il pour une femme en 3eme classe de plus de 30 ans ?

<details>
<summary>👉 Cliquez ici pour voir la réponse</summary>

**Réponse :** Elle est probablement décédée (case orange en bas à droite) --> discrimination sociale ?
</details>

Le modèle utilise des critères comme le sexe ou la classe sociale pour décider de la vie ou de la mort. Est-ce que cela signifie que le modèle est 'sexiste' ou 'classiste' intrinséquement ou est-ce qu'il ne fait que refléter la réalité historique, et donc le biais, des données ?

In [ ]:
#@title Simulation : Auriez-vous survécu au Titanic ?

# Saisie des données via le formulaire à droite
sexe = "femme" #@param ["homme", "femme"]
age = 25 #@param {type:"slider", min:1, max:100, step:1}
classe = 1 #@param [1, 2, 3]
freres_soeurs = 0 #@param {type:"number"}
parents_enfants = 0 #@param {type:"number"}
prix_billet = 500 #@param {type:"number"}

# Conversion des données pour le modèle
sexe_code = 1 if sexe == "femme" else 0

In [ ]:
# Création d'un petit tableau avec vos données entrées
moi = pd.DataFrame([[classe, sexe_code, age, freres_soeurs, parents_enfants, prix_billet]],
                  columns=['Classe', 'Sexe', 'Age', 'Frères_Sœurs_Époux', 'Parents_Enfants', 'Prix_Billet'])

In [ ]:
# Prédiction
survie = model.predict(moi)[0]
probabilite = model.predict_proba(moi)[0][1]

# Résultat
if survie == 1:
    print(f"✅ Félicitations ! Vous auriez SURVÉCU (Probabilité : {probabilite*100:.1f}%)")
else:
    print(f"❌ Désolé... Vous n'auriez PAS survécu (Probabilité de survie : {probabilite*100:.1f}%)")

#6. Un modèle plus avancé : RandomForest

Pourquoi changer de modèle ? Pensez à un arbre de décision comme consulter l'avis d'un seul expert qui parrcourt le tableau des données. Il peut se tromper ou être trop rigide dans sa sélection de paramètres discriminants.

Une forêt aléatoire c'est comme demander l'avis à 100 experts différents qui regardent chacun une seule partie du problème puis de faire un vote entre eux. La majorité l'emporte.

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
rf_predictions = rf_model.predict(X_eval)
rf_score = accuracy_score(y_eval, rf_predictions)

In [ ]:
print(f"Précision de l'arbre de décision : {score * 100:.2f}%")
print(f"Précision de la forêt aléatoire : {rf_score * 100:.2f}%")

L'avantage avec ces modèles, c'est qu'on facilement mesurer les paramètres qui ont eu une importance dans leur calcul :

In [ ]:
# Récupérer l'importance des colonnes
importances = pd.Series(rf_model.feature_importances_, index=features)
importances = importances.sort_values(ascending=False)

In [ ]:
# Affichage
plt.figure(figsize=(10,6))
sns.barplot(x=importances, y=importances.index)
plt.title("Quels facteurs ont le plus compté pour le modèle ?")
plt.xlabel("Score d'importance")
plt.show()

In [ ]:
#Matrice de confusion pour comprendre les erreurs du modèle
cm = confusion_matrix(y_eval, predictions)
plt.figure(figsize=(8,6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Décédé', 'Survécu'])
disp.plot(cmap='Blues', values_format='d')
plt.title("Où le modèle se trompe-t-il ?")
plt.show()

Comment lire une matrice de confusion binaire ? Attention, pas évident aux premiers abords :

- Vrais Négatifs (Haut-Gauche) : Le passager est mort et le modèle a prédit qu'il mourrait, c'est parfait.

- Vrais Positifs (Bas-Droite) : Le passager a survécu et le modèle a prédit qu'il survivrait, c'est parfait.

- Faux Positifs (Haut-Droit) : Le modèle a dit "Il va survivre" mais il est mort, c'est une erreur.

- Faux Négatifs (Bas-Gauche) : Le modèle a dit "Il va mourir" mais il a survécu, c'est une erreur.

Nous avons vu la précision du modèle et sa matrice de confusion mais si le modèle se trompe donc, peut-on savoir un peu plus précisément *comment* ?

Ci-dessous un rapport de classification standard :

In [ ]:
print(classification_report(y_eval, rf_predictions, target_names=['Décédé', 'Survécu']))

In [ ]:
#Je vous remets le rapport de l'arbre :
print(classification_report(y_eval, predictions, target_names=['Décédé', 'Survécu']))

## Comment lire le rapport

- Précision (la "fiabilité") : Quand le modèle dit "Ce passager va survivre", à quel point pouvez-vous lui faire confiance ?

Si la précision est de 0.80, ça veut dire que sur 100 personnes prédites "Survivantes", 80 le sont vraiment (et 20 sont en fait décédées).

- Rappel (la "détection") : Sur tous les gens qui ont réellement survécu, combien le modèle a-t-il réussi à en retrouver ?

Si le rappel est de 0.60, ça veut dire que le modèle a "oublié" 40% des survivants, il les a déclarés mort, à tort. (car nous avons une classification binaire)

- F1-Score (le "compromis") : C'est la moyenne des deux. Si vous voulez un modèle équilibré qui ne fait ni trop de fausses joies, ni trop d'oublis, c'est ce chiffre qu'il faut regarder.

----------------------------

# Pour la suite :
- XGBoost : peut être plus précis que RFC mais sensible à l'overfitting
- Feature engineering : modifier les features pour améliorer les paramètres disponibles (extraction du titre devant le nom de la personne, additionner la taille de la famille, ajouter le port d'embarcation, etc.)
- Modification des hyperparamètres : 100 arbres, 1000 arbres, etc. ?
- Validation croisée : au lieu de tester une fois sur 20% des données, on fait 5 découpages différents pour 5 expériences différentes et on évalue 5 fois. En prenant la moyenne des évaluations, on obtiendra un score plus fiable.

## Question :
Est-ce que la taille de la famille a un impact sur les chances de survie ?

<details>
<summary>👉 Cliquez ici pour voir la réponse</summary>

**Réponse :**
> df['Taille_Famille'] = df['Frères_Sœurs_Époux'] + df['Parents_Enfants'] + 1
</details>

In [ ]:
#Indice : ajouter une nouvelle colonne dans le DataFrame à partir d'une addition de colonnes

## Question :
Si on découvre que 100% des gens qui ont embarqué à Cherbourg ont survécu, est-ce parce que l'air de Cherbourg rend immortel ?

<details>
<summary>👉 Réponse (Le concept de corrélation)</summary>
Non ! C'est probablement parce que la majorité des passagers riches (1ère classe) ont embarqué à Cherbourg. Ce n'est pas le port d'embarcation qui sauve mais la classe sociale associée au port. Leçon primordiale : **corrélation n'est pas causalité**. C'est toujours important à garder à l'esprit en analysant des données et des résultats.
</details>

# Tentez donc d'avoir le meilleur score !

Il existe même une compétition permanente pour vous exercer : https://www.kaggle.com/c/titanic